In [6]:
import psycopg2

def make_connection():

    host = 'localhost'  
    database = 'secure environment'
    user = 'postgres'
    password = '****'
    port = '5432'  

    try:
        # Establish a connection to the database
        connection = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port=port
        )
    
        # Create a cursor object to interact with the database
        cursor = connection.cursor()

        # Execute a sample query
        cursor.execute('SELECT version();')
        db_version = cursor.fetchone()
        print(f"Connected to: {db_version[0]}")

    except Exception as e:
        print(f"Error: {e}")
    return connection

   

In [7]:
from keras_facenet import FaceNet

face_model = FaceNet()


In [8]:
from ultralytics import YOLO
yolo_model = YOLO("yolov8n.pt")

In [9]:

from sklearn.metrics.pairwise import cosine_similarity

# Function to compare face embeddings (cosine similarity)
def get_similarity(embedding1, embedding2):
    embedding1 = np.reshape(embedding1, (1, -1))  
    embedding2 = np.reshape(embedding2, (1, -1)) 
    similarity = cosine_similarity(embedding1, embedding2)[0][0]  # Get the similarity value
    return similarity 

In [ ]:
import cv2
import numpy as np
from PIL import Image
import ast

# Assuming these functions and models are already initialized
model = yolo_model
con = make_connection()

# Open a connection to the webcam (0 is the default webcam)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Cannot access the webcam")
    exit()

# Function to pad the embedding
def pad_embedding(embedding, target_size=512):
    embedding = np.array(embedding, dtype=np.float32)
    current_size = len(embedding)
    
    if current_size > target_size:
        return embedding[:target_size]  # Trim if too large
    elif current_size < target_size:
        padding = np.zeros(target_size - current_size, dtype=np.float32)
        return np.concatenate((embedding, padding))
    return embedding

# Function to get the face embedding
def get_face_embedding(image, target_size=512):
    face_image_pil = Image.fromarray(image).resize((160, 160))  # Resize to 160x160
    face_image_array = np.array(face_image_pil)
    embeddings = face_model.embeddings([face_image_array])[0]
    
    # Debugging: Ensure embeddings are numeric
    print("Embedding type:", type(embeddings))
    print("Embedding size:", len(embeddings))
    
    padded_embedding = pad_embedding(embeddings, target_size)
    return padded_embedding

# Process frames from the webcam in real-time
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Perform face detection
    results = model(frame)
    
    cursor = con.cursor()
    cursor.execute("SELECT name, emb_vector FROM face_embedding")
    rows = cursor.fetchall()

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls.item())
        confidence = box.conf.item()
        label = model.names[class_id]
        
        person_reg = frame[y1:y2, x1:x2]
        # Approximate face region
        face_roi = person_reg[0:(y2 - y1)//2, :]  
        
        embedded_face = get_face_embedding(face_roi, target_size=512)
        if class_id != 67:
            if confidence > 0.7 and class_id == 0: 
                for row in rows:
                    embedding = row[1]
                    
                    # Convert string to numeric array if needed
                    if isinstance(embedding, str):
                        embedding = ast.literal_eval(embedding)
                        embedding = np.array(embedding, dtype=np.float32)
                    
                    similarity = get_similarity(embedded_face, np.array(embedding))
                    print(f"Similarity: {similarity}")
                    
                    if similarity > 0.7:
                        color = (0, 255, 0)
                        name = row[0]
                        print(f"Recognized: {name}")
                        break
                else:
                    name = "Unknown"
                    color = (0, 0, 255)
                    
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f'{name}: {confidence:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        else:
            cv2.putText(frame, "Using cell phone", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the output frame
    cv2.imshow('YOLOv8 Real-Time Face Detection', frame)

    # Break on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Connected to: PostgreSQL 16.3, compiled by Visual C++ build 1939, 64-bit

0: 480x640 1 person, 1061.2ms
Speed: 250.0ms preprocess, 1061.2ms inference, 66.6ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Embedding type: <class 'numpy.ndarray'>
Embedding size: 512
Similarity: -0.13527243114930249
Similarity: -0.030507370059122758
Similarity: -0.030507370059122758
Similarity: -0.030507370059122758
Similarity: -0.10412113472591794
Similarity: -0.10412113472591794
Similarity: -0.030507360337416158
Similarity: -0.17159751573879872

0: 480x640 1 person, 497.7ms
Speed: 92.4ms preprocess, 497.7ms inference, 57.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step
Embedding type: <class 'numpy.ndarray'>
Embedding size: 512
Similarity: -0.13827416689088418
Similarity: 0.009939527984210448
Similarity: 0.009939527984210448
Similarity: 0.009939527984210448
Similarity: -0.04683581666861013
Similarity: -0.04683581666861013
Sim

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - Can't parse 'org'. Input argument doesn't provide sequence protocol
>  - Can't parse 'org'. Input argument doesn't provide sequence protocol
